In [ ]:
import os
import shutil

base_dir = "/content/tiny-imagenet-200"
val_dir = os.path.join(base_dir, "val")
images_dir = os.path.join(val_dir, "images")
ann_file = os.path.join(val_dir, "val_annotations.txt")

# Read annotations
with open(ann_file) as f:
    annotations = [line.strip().split('\t') for line in f]

# Create class folders and move images
for img, cls, *_ in annotations:
    cls_dir = os.path.join(val_dir, cls)
    os.makedirs(cls_dir, exist_ok=True)
    shutil.move(
        os.path.join(images_dir, img),
        os.path.join(cls_dir, img)
    )

os.rmdir(images_dir)

In [ ]:
import os
os.environ["TF_USE_LEGACY_KERAS"] = "1"

!pip -q install tf-keras

In [ ]:
import tensorflow as tf
import tensorflow.keras.layers as layers

In [ ]:
# load training set
train_ds = tf.keras.utils.image_dataset_from_directory(
    "/content/tiny-imagenet-200/train",
    image_size=(256, 256),
    batch_size=None,
    label_mode="int"
)

# load test set
test_ds = tf.keras.utils.image_dataset_from_directory(
    "/content/tiny-imagenet-200/val",
    image_size=(224, 224),
    batch_size=64,
    label_mode="int"
)

Found 100000 files belonging to 200 classes.
Found 10000 files belonging to 200 classes.


In [ ]:
def random_crop(image):
  image = tf.image.random_crop(image, (224, 224, 3))
  image = tf.image.random_flip_left_right(image)
  return image

In [ ]:
train_ds = train_ds.map(lambda x, y: (random_crop(x), y))
train_ds = train_ds.batch(64)

In [ ]:
def normalisation(images):
  mean = tf.constant([0.5, 0.5, 0.5])
  std = tf.constant([0.5, 0.5, 0.5])
  return (images / 255.0 - mean) / std

In [ ]:
train_ds = train_ds.map(lambda x, y: (normalisation(x), y))
test_ds = test_ds.map(lambda x, y: (normalisation(x), y))

In [ ]:
def tonchw(image, label):
  return tf.transpose(image, [0, 3, 1, 2]), label

In [ ]:
train_ds = train_ds.map(tonchw)
train_ds = train_ds.prefetch(tf.data.AUTOTUNE)
test_ds = test_ds.map(tonchw)
test_ds = test_ds.prefetch(tf.data.AUTOTUNE)

In [ ]:
from transformers import TFViTModel

vit = TFViTModel.from_pretrained(
    "google/vit-base-patch16-224-in21k",
    from_pt=True,
    use_safetensors=False
)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
TensorFlow and JAX classes are deprecated and will be removed in Transformers v5. We recommend migrating to PyTorch classes or pinning your version of Transformers.
All PyTorch model weights were used when initializing TFViTModel.

All the weights of TFViTModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFViTModel for predictions without further training.


In [ ]:
inputs = layers.Input(shape=(3, 224, 224))

outputs = vit(inputs, training=True)
cls_token = outputs.last_hidden_state[:, 0, :]
x = layers.Dropout(0.1)(cls_token)
x = layers.Dense(200)(x)

model = tf.keras.Model(inputs, x)

In [ ]:
vit.trainable = False

model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-3),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=["accuracy"]
)

model.fit(train_ds, epochs=5)

Epoch 1/5
1563/1563 [==============================] - 580s 363ms/step - loss: 1.3280 - accuracy: 0.8003
Epoch 2/5
1563/1563 [==============================] - 568s 363ms/step - loss: 0.6213 - accuracy: 0.8518
Epoch 3/5
1563/1563 [==============================] - 568s 363ms/step - loss: 0.5504 - accuracy: 0.8633
Epoch 4/5
1563/1563 [==============================] - 568s 363ms/step - loss: 0.5033 - accuracy: 0.8709
Epoch 5/5
1563/1563 [==============================] - 568s 363ms/step - loss: 0.4677 - accuracy: 0.8787


In [ ]:
vit.trainable = True

model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-5),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=["accuracy"]
)

model.fit(train_ds, epochs=20)

Epoch 1/20


1563/1563 [==============================] - 1614s 1s/step - loss: 0.3473 - accuracy: 0.9070
Epoch 2/20
1563/1563 [==============================] - 1566s 1s/step - loss: 0.2485 - accuracy: 0.9315
Epoch 3/20
1563/1563 [==============================] - 1566s 1s/step - loss: 0.1883 - accuracy: 0.9485
Epoch 4/20
1563/1563 [==============================] - 1566s 1s/step - loss: 0.1470 - accuracy: 0.9600
Epoch 5/20
1563/1563 [==============================] - 1566s 1s/step - loss: 0.1109 - accuracy: 0.9709
Epoch 6/20
1563/1563 [==============================] - 1566s 1s/step - loss: 0.0854 - accuracy: 0.9787
Epoch 7/20
1563/1563 [==============================] - 1566s 1s/step - loss: 0.0648 - accuracy: 0.9848
Epoch 8/20
1563/1563 [==============================] - 1566s 1s/step - loss: 0.0506 - accuracy: 0.9888
Epoch 9/20
1563/1563 [==============================] - 1566s 1s/step - loss: 0.0404 - accuracy: 0.9914
Epoch 10/20
1563/1563 [==============================] - 1566s 1s/step - lo

In [ ]:
model.evaluate(test_ds)

157/157 [==============================] - 60s 363ms/step - loss: 0.5162 - accuracy: 0.8987


[0.5162237882614136, 0.8986999988555908]